# Application My Credit

>> Descriptions détaillées des colonnes données clients bancaires :

1 - âge (numérique)

2 - emploi : type d'emploi (catégorique : "admin.", "inconnu", "chômeur", "gestion", "femme de ménage", "entrepreneur", "étudiant", "col bleu", "indépendant", "retraité", "technicien", "services")

3 - matrimonial : état civil (catégorique : "marié", "divorcé", "célibataire" ; attention : "divorcé" signifie divorcé ou veuf)

4 - éducation (catégorique : "inconnu", "secondaire", "primaire", "tertiaire")

5 - défaut : le crédit est-il en défaut ? (binaire : "oui", "non")

6 - solde : solde annuel moyen, en euros (numérique) 

7 - logement : a-t-il un prêt logement ? (binaire : "oui", "non")

8 - prêt : avez-vous un prêt personnel ? (binaire : "oui", "non")

>> lié au dernier contact de la campagne en cours :

9 - contact : type de communication du contact (catégorique : "inconnu", "téléphone", "cellulaire")

10 - jour : dernier jour de contact du mois (numérique)

11 - mois : dernier mois de contact de l'année (catégorique : "jan" , "feb", "mar", …, "nov", "dec")

12 - durée : durée du dernier contact, en secondes (numérique)

>> autres attributs :

13 - campagne : nombre de contacts effectués pendant cette campagne et pour cette client (numérique, inclut le dernier contact)

14 - pdays : nombre de jours écoulés après que le client a été contacté pour la dernière fois lors d'une campagne précédente (numérique, -1 signifie que le client n'a pas été contacté auparavant)

15 - précédent :nombre de contacts effectués avant cette campagne et pour ce client (numérique)

16 - poutcome : résultat de la campagne marketing précédente (catégorique : "inconnu", "autre", "échec", "succès")

>> Variable de sortie (cible souhaitée) :

17 - y - le client a-t-il obtenu un crédit ? (binaire : "oui", "non")

In [6]:
!pip install pandas


'pip' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [1]:
import pandas as pd
import numpy as np

df_train = pd.read_csv('train.csv', delimiter=';')
df_test = pd.read_csv('test.csv', delimiter=';')
df_train.shape

(45211, 17)

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

In [4]:
X_train = df_train.drop("y", axis=1)
y_train = df_train["y"]
X_test = df_test.drop("y", axis=1)
y_test = df_test["y"]

In [5]:
cat_list = []
cont_list = []
for column in X_train:
    if X_train[column].dtype == "object":
        cat_list.append(column)
    else:
        cont_list.append(column)

In [8]:
len(X_train.columns)

16

In [9]:
ct = ColumnTransformer(
    [("Onehot", OneHotEncoder(), cat_list),
     ("Normalizer", StandardScaler(), cont_list)]
)
X_train = ct.fit_transform(df_train)
X_test = ct.transform(df_test)

In [16]:
import joblib

foret = joblib.load("foret.sav")

In [17]:
data = {
    "age": 30,
    "job": "admin.",
    "marital": "married",
    "education": "secondary",
    "default": "no",
    "housing": "yes",
    "balance": 5000.0,
    "previous": 2,
    "loan": "no",
    "contact": "cellular",
    "day": 15,
    "month": "may",
    "duration": 180,
    "campaign": 3,
    "pdays": 15,
    "poutcome": "success"
}
data = pd.DataFrame(data, index=[0])
data = ct.transform(data)
pred = foret.predict(data)

In [18]:
pred

array(['no'], dtype=object)

In [57]:
forest = RandomForestClassifier()
forest.fit(X_train, y_train)

RandomForestClassifier()

In [58]:
from sklearn.metrics import accuracy_score

In [59]:
y_pred = forest.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

1.0


In [64]:
import joblib
joblib.dump(forest, "foret.sav")
joblib.dump(ct, 'transformer.pkl')

['transformer.pkl']